案例介绍
- 案例背景：以某大型电商平台的用户行为数据为数据集，使用大数据处理技术分析海量数据下的用户行为特征，并通过建立分类模型对用户行为做出预测；
- 案例思路：
    - 使用大数据处理技术读取海量数据
    - 海量数据预处理
    - 抽取部分数据调试模型
    - 使用海量数据搭建模型

In [1]:
#全部行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

数据字典：  
U_Id：the serialized ID that represents a user  
T_Id：the serialized ID that represents an item  
C_Id：the serialized ID that represents the category which the corresponding item belongs to  
Ts：the timestamp of the behavior  
Be_type：enum-type from ('pv', 'buy', 'cart', 'fav')  
> pv: Page view of an item's detail page, equivalent to an item click  
> buy: Purchase an item  
> cart: Add an item to shopping cart  
> fav: Favor an item

# 读取数据

这里关键是使用dask库来处理海量数据，它的大多数操作的运行速度比常规pandas等库快十倍左右。

pandas在分析结构化数据方面非常的流行和强大，但是它最大的限制就在于设计时没有考虑到可伸缩性。pandas特别适合处理小型结构化数据，并且经过高度优化，可以对存储在内存中的数据执行快速高效的操作。然而随着数据量的大幅度增加，单机肯定会读取不下的，通过集群的方式来处理是最好的选择。这就是Dask DataFrame API发挥作用的地方：通过为pandas提供一个包装器，可以智能的将巨大的DataFrame分隔成更小的片段，并将它们分散到多个worker(帧)中，**并存储在磁盘中而不是RAM中**。

Dask DataFrame会被分割成多个部门，每个部分称之为一个分区，每个分区都是一个相对较小的DataFrame，可以分配给任意的worker，并在需要复制时维护其完整数据。具体操作就是**对每个分区并行**或单独操作(多个机器的话也可以并行)，然后再将结果合并，其实从直观上也能推出Dask肯定是这么做的。

In [2]:
# 安装库（清华镜像）
# pip install dask -i https://pypi.tuna.tsinghua.edu.cn/simple

In [2]:
import gc   # 垃圾回收接口
import dask   # 并行计算接口
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import dask.dataframe as dd   # dask中的数表处理库
import sys   # 外部参数获取接口

面对海量数据，跑完一个模块的代码就可以加一行gc.collect()来做内存碎片回收  

In [3]:
gc.collect()

15

Dask Dataframes与Pandas Dataframes具有相同的API

与pandas不同，这里我们仅获取数据框的结构，而不是实际数据框。Dask已将数据帧分为几块加载，这些块存在于磁盘上，而不存在于RAM中。如果必须输出数据帧，则首先需要将所有数据帧都放入RAM，将它们缝合在一起，然后展示最终的数据帧。使用.compute()强迫它这样做，否则它不.compute() 。其实dask使用了一种延迟数据加载机制，这种延迟机制类似于python的迭代器组件，只有当需要使用数据的时候才会去真正加载数据。

In [4]:
# 读取数据


,U_Id,T_Id,C_Id,Be_type,Ts
npartitions=58,,,,,
,int64,int64,int64,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


# 数据预处理

## 数据压缩

In [7]:
types = {'U_Id':'uint32',
         'T_Id':'uint32',
         'C_Id':'uint32',
         'Be_type':'object',
         'Ts':'int64'
}


## 缺失值

Dask Series Structure:
npartitions=1
Be_type    int64
U_Id         ...
dtype: int64
Dask Name: dataframe-sum-agg, 233 tasks

## 数据探索与可视化

除了matplotlib和seaborn，我们还可以使用pyecharts库。  
pyecharts是一款将python与百度开源的echarts结合的数据可视化工具。新版的1.X和旧版的0.5.X版本代码规则大不相同，新版详见官方文档https://gallery.pyecharts.org/#/README

In [24]:
# pip install pyecharts -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


### 饼图

In [23]:
# 这里我们以漏斗图为例，其余的图请同学们自己完成


### 漏斗图

### 涟漪散点图

### 象形柱图

### 多维散点图

### 时间轴

### 词云图

# 时间戳问题

dask对于时间戳的支持非常不友好

# 特征工程

思路：不考虑时间窗口，只以用户的点击和收藏等行为来预测是否购买  
流程：以用户ID(U_Id)为分组键，将每位用户的点击、收藏、加购物车的行为统计出来，分别为
> 是否点击，点击次数；是否收藏，收藏次数；是否加购物车，加购物车次数  

以此来预测最终是否购买

In [24]:
# 去掉时间戳


## 行为类型

In [32]:
from collections import Counter   # 计数器包

### 点击次数

### 加购次数

### 收藏次数

## 相关分析

是否加购与加购次数、是否收藏与收藏次数之间存在一定相关性，但经验证剔除其中之一与纳入全部变量效果基本一致，故之后使用全部变量建模。

## 数据标签

In [98]:
import seaborn as sns

In [25]:
#是否购买（Label）


# 建立模型

## 划分数据集

In [102]:
from sklearn.model_selection import train_test_split

## 分类模型

### 模型建立

In [43]:
from sklearn.linear_model import LogisticRegression

In [26]:
#简单测试


### 模型评估

In [46]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import auc,roc_curve

In [27]:
#分类报告


In [28]:
#roc曲线
